In [3]:
import pandas as pd
import numpy as np
import time 
import os
from gen_ball import gen_balls
from classGBTSVM import OvO_GBTSVM, GBTSVM
from classLGBTSVM import LGBTSVM, OvO_LGBTSVM, OvR_LGBTSVM

In [4]:
directory = r"D:/LGBTSVM_MCD/LGBTSVM-MCD/Dataset"
file_path = os.path.join(directory, "chess_krvkp.csv")
file_data = np.loadtxt(file_path, delimiter=',')
m, n = file_data.shape
for i in range(m):
    if file_data[i, n-1] == 0:
        file_data[i, n-1] = -1

In [ ]:
file_data

In [3]:
np.random.seed(0)
indices = np.random.permutation(m)
file_data = file_data[indices]
A_train=file_data[0:int(m*(1-0.20))]
A_test=file_data[int(m * (1-0.20)):]

In [4]:
A_train

array([[ 2.81956 , -0.275152, -0.197483, ...,  0.602317, -0.572443,
         1.      ],
       [-0.354555,  3.63322 , -0.197483, ...,  0.602317,  1.74635 ,
         1.      ],
       [-0.354555, -0.275152, -0.197483, ...,  0.602317, -0.572443,
         1.      ],
       ...,
       [ 2.81956 , -0.275152, -0.197483, ..., -1.65973 , -0.572443,
         1.      ],
       [-0.354555, -0.275152, -0.197483, ...,  0.602317, -0.572443,
         1.      ],
       [-0.354555,  3.63322 , -0.197483, ..., -1.65973 , -0.572443,
        -1.      ]])

In [5]:
pur = 1 - (0.015 * 5)          
# pur = 1 - (0.015 * 10)                
num = 4
c1=0.00001
c2=0.00001
            
A_train = gen_balls(A_train, pur=pur, delbals=num)

Radius=[]
for i in A_train:
    Radius.append(i[1])
Center=[]
for ii in A_train:
    Center.append(ii[0])
Label=[]
for iii in A_train:
    Label.append(iii[-1])
Radius=np.array(Radius)
Center=np.array(Center)
Label=np.array(Label)
Z_train=np.hstack((Center,Radius.reshape(Radius.shape[0], 1)))
Lab=Label.reshape(Label.shape[0], 1)
A_train=np.hstack((Z_train,Lab))

In [ ]:
A_train

In [6]:
gbtsvm = GBTSVM(d1=0.1, d2=0.1, eps1=0.05, eps2=0.05)
lgbtsvm = LGBTSVM(d1=0.1, d2=0.1, d3=0.1, d4=0.1)
gbtsvm.fit(A_train)
lgbtsvm.fit(A_train)
y_pred_gbtsvm = gbtsvm.predict(A_test)
y_pred_lgbtsvm = lgbtsvm.predict(A_test)

accuracy_gbtsvm = gbtsvm.score(A_test)
accuracy_lgbtsvm = lgbtsvm.score(A_test)

In [7]:
accuracy_gbtsvm,accuracy_lgbtsvm

(88.75, 91.40625)

In [8]:
X_train = A_train[:,:-1]
X_test = A_test[:,:-1]
y_train = A_train[:,-1]
y_test = A_test[:,-1]

In [9]:
ovo_tsvm = OvO_GBTSVM(d1=0.1, d2=0.1, eps1=0.05, eps2=0.05)

# Huấn luyện mô hình
ovo_tsvm.fit(X_train, y_train)
y_pred = ovo_tsvm.predict(X_test)
print(y_test)
# Đánh giá độ chính xác
accuracy = ovo_tsvm.score(X_test, y_test)

accuracy

[ 1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1.
 -1.  1.  1.  1. -1.  1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1. -1.
 -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1.  1.  1. -1. -1. -1.  1.
  1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1. -1.  1. -1. -1. -1.  1.
 -1.  1.  1.  1. -1.  1. -1.  1. -1.  1.  1.  1.  1. -1.  1. -1. -1. -1.
 -1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1. -1. -1.  1. -1.  1.  1.  1.
 -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1.
 -1.  1.  1. -1. -1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1. -1.
 -1. -1.  1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1.
  1. -1. -1.  1. -1.  1.  1.  1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1.
 -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1.  1.  1.  1. -1.
  1. -1.  1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1.  1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.  1

88.75

In [10]:
ovo_tsvm = OvO_LGBTSVM(d1=0.1, d2=0.1, d3=0.1, d4=0.1)

# Huấn luyện mô hình
ovo_tsvm.fit(X_train, y_train)
y_pred = ovo_tsvm.predict(X_test)
print(y_test)
# Đánh giá độ chính xác
accuracy = ovo_tsvm.score(X_test, y_test)

accuracy

[ 1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1.
 -1.  1.  1.  1. -1.  1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1. -1.
 -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1.  1.  1. -1. -1. -1.  1.
  1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1. -1.  1. -1. -1. -1.  1.
 -1.  1.  1.  1. -1.  1. -1.  1. -1.  1.  1.  1.  1. -1.  1. -1. -1. -1.
 -1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1. -1. -1.  1. -1.  1.  1.  1.
 -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1.
 -1.  1.  1. -1. -1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1. -1.
 -1. -1.  1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1.
  1. -1. -1.  1. -1.  1.  1.  1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1.
 -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1.  1.  1.  1. -1.
  1. -1.  1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1.  1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.  1

0.9140625

In [11]:
ovo_tsvm = OvR_LGBTSVM(d1=0.1, d2=0.1, d3=0.1, d4=0.1)

# Huấn luyện mô hình
ovo_tsvm.fit(X_train, y_train)
y_pred = ovo_tsvm.predict(X_test)
print(y_test)
# Đánh giá độ chính xác
accuracy = ovo_tsvm.score(X_test, y_test)

accuracy

[ 1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1.
 -1.  1.  1.  1. -1.  1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1. -1.
 -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1.  1.  1. -1. -1. -1.  1.
  1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1. -1.  1. -1. -1. -1.  1.
 -1.  1.  1.  1. -1.  1. -1.  1. -1.  1.  1.  1.  1. -1.  1. -1. -1. -1.
 -1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1. -1. -1.  1. -1.  1.  1.  1.
 -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1.
 -1.  1.  1. -1. -1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1. -1.
 -1. -1.  1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1.
  1. -1. -1.  1. -1.  1.  1.  1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1.
 -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1.  1.  1.  1. -1.
  1. -1.  1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1.  1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.  1

0.9140625